In [1]:
import numpy as np
#import tensorflow as tf
from basenji import seqnn
import json
with open('cs273b_params.json') as params_open:
    params = json.load(params_open)
params_model = params['model']
params_train = params['train']

#model = seqnn.SeqNN(params_model)

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
import keras
model = keras.models.load_model('final_model.h5')

Using TensorFlow backend.


RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.

In [57]:
def eval_seqs(seqs, model, target_ind):
    activations = np.squeeze(model.predict(seqs), axis=1)
    print(activations.shape)
    diffs = activations[:,target_ind] - (activations[:,(target_ind+1)%4] + activations[:,(target_ind+2)%4] + activations[:,(target_ind+3)%4])/3
    return activations, diffs, np.mean(diffs)


In [65]:
for i in range(4):
    seqs = np.load('cell'+ str(i) + '_seqs.npy')[:10]
    print(seqs.shape)
    #seqs = np.squeeze(seqs, axis=3)
    activations, diffs, mean_diff = eval_seqs(seqs, model, i)
    np.save('activations_'+str(i)+'.npy',activations)
    print(mean_diff)
    print(np.mean(activations[:,i]))

(10, 1000, 4)
(10, 4)
-0.18180875
0.67436427
(10, 1000, 4)
(10, 4)
-0.1059802
0.7640686
(10, 1000, 4)
(10, 4)
-0.535961
0.38931817
(10, 1000, 4)
(10, 4)
0.8198916
1.4281147


In [22]:
def encode_seq(seq):
    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    arr = np.zeros((len(seq), 4))
    for i in range(len(seq)):
        b = seq[i]
        if b in mapping.keys():
            arr[i, mapping[b]] = 1
        else:
            arr[i,:] = 1
    return arr

In [23]:
strs = ['GGACTATAATTAGTACATTATTGAATAGCCAAATATTTAAAAATAGGTGAACTTAATTATCTAGATTGGTAGGCTAAATAAAAAAAAAATAAAATTTATCGATAAATAGCTTGTGTCATTCTAAGATAAATACAGAAATTTATTTTTATTAATTATATAAGATCAACTAAACTCTATTATAAGTTTAAATACTTGGCTAAATTCGGAGATCCACAATGTTTTATTTTGCAATTATTTTTTAACGTCTAATTAAATAGCCAATCACAGAACTTATATCATGAAATTATAGAATGGGGATAGATGATAGCATTAATACAGCTTTAGGCACTAGAAAAGCTCCAAAGAAAATGCCAGGATTTAAGTAATTTAAATATGTAGGTCATATCGCAACTTTATTATTAGGAGTCGGAGAAATGGTATAAACAATCGATTTAGATTATGAGAGGTCGAGAGTAATATATTAAAAGAGAAGAATTAGCCATGTAAATATTTATTAAAAAATTAATATAACAATACTAAGACAGCATAGATAGCAGTAATATAGTACTATAGCTACAGTATATATTATATGTGACCTATCTAAATTTGCGAATGATATCACTTAAACAAGAATGGAGTGTTAAAATTTGAATAACAGCTAGAAAATATATAATAAACAAAGAAGATTATTAAAGATGCAATGAAATTAGCTATATTATAAAATTAAATAAATTGTTGATCTATATTAATAATTATTATCAAAGTAGAATATTAGGACTAGAATGGAAGTTAGATTTGTGAAAATTCATTTATATAATACAGTAGAATAGATAATGTGTGTAATCTAAAAATGAATGAAGATAAATTAAAAGCAACTATAATAGTGACTTAGAGATCTTCTTGATCACTATGATAGCACAATTAGGAATTTTTGCTAGACGATAATGAACTTTAAACTAGTGAATAAATATTTTTACAGATAATATGAATTCTTGCTAAGGAATATTAGGATTGCCACATA',
        'AAAAAAATCGCGCAATACGACTGGACTAGTTGATTTAAAAAACCGAATAAAAAGTAAAGCGTCAGAACACTATTAAAAGAAACTTACCGAAAATAACTAAAAAAATCAAATAACAACGGCACACGGTTTAAAATAAAAACCAACCAGTAATTCGTAACCTTAATCCGGAAATTTAGAATAGTAAAAAAAAAAAAAGGAAACTAACCCAAAAGTACGTTTAACCAGACTAAACGCGGAAATCAAACGAGATGCTATAAAAAAAAGAAAAGTACGTAACTCACCATAACCCGTATAAAACAGTTATTTAAAAACCAAGTAACAAGCGTAAAAAAACCTAAAGCCTTTTTATAAAACATACAGAGATTCGCCTCAAAACGTATACTTAAAAACCGATAAAAAACGCATTTCAGAAAGCTTCCTACATTACCACCGAAACGAAGTAGTAAAAACAGCACTAAAAAAATAAAAAAACAAAAAAGAAGCGCAAACACCCTCTACGTACCAAAAAAACCGCCAAACGATCATTACGGCCAATTCAGATTCAAACAAACAACACTAAAAAAAAATTTTAAACTTACAATGTACCTTACCTAATTACATCTAACTAAGTACTGAACAAAAAAACAAAACACCACACTACACATACCGACCGTCCGTTTGAAAAAAAAAACAATTAAAAACGAAAGAAAACGTTAAACCATTAAACATTAACTGACCAAAAAAACTCCAAAAATTACGACAAAAAACGAAAAAACGTAGTTTATTGTAGACCCCCCCACACCCGCTATCTACGGCGCACAATAGAATTCCTTTTAGAAACTAAAAAGATAAAAAAAAGCATTCAAAAACAGAATGGGGCTTTACAAAATAAGTAAAGTAGCCAAACGATATTAAACAAAACCATTTCGTTAGCAATACCCTAATCTCACTTAATTATAAATACGAACTTTTTACGTTTACCGAAACCCCCAACTGCAGTACTACTTAAAGCGATGCAAAC',
        'TTGAACGTAATACCTCATAGAAGTAATTAATGATTATCTGTTGTGGCCGGGAAGGATAATAGAAAGACTTTTTATATAATAATTTAAAATGTAGCTGTAAAAGTATACTTTGGCCGAAATCAAAAATTTAATTTTTTTGATCTTACATTTCTAAGGTATTTTTTTATTTAATATTTTTTGTTTCTTATTTTTTTTTTACCTTGAAACTCCTGTAAGTTATCTTTTAAATTTTTTATAAGGTTCGTTGTTTTAATTATTTTTGTATGTGTACGAGATTGTATTAAATGTAGTTTATCTTGTTTTCCCTTGGGAGTTTAATCTCTAGCTACCGTCGTTGAATTGCACACTTGTCCCACTATTATTGTCATTGAAACTAATAATAATTCAAGGCTGGTATATGGGAGAGTTGAGCTTTTTGATCTCTAGCTATTCTAATTTTAATTAATATAGTATATAGAGAGCTAGTAAGAGTGCATAATCATGCTATACTAGTGAGTGTGAGTATATTATTAAGTAAGTTCTATTCTAGTATAAATATGGCTTTTTTTGATATCCAATAGATGAGAACTGAGAGAAACGTCTCTGTTATCATGGCTAGTAAGATAAAGTAGCCCAATGACTAGTAATTTCAATTTCAAGTTCGCGGCACATTTCTCCCATTTCGTAAATTAACGTATATTAAAAATTTCTGGCTATTTTATTATATTTATATCTTCCCGACAACACATGGATTTTTTTCCTGATCACCAAACTCTTATTTTTTTTTTTATCTGTGCTATTTAAAAAACTAATTTTTGTGAGTATATTTGATTGAATAGTTCTGATCTATTATGGATCTTTCTACACTTAGTTGTTTTATCGGATTTATTTAAGAGGCAATTAAAAGAGACAGATTTGATGTTCTGACTTTGACTCTTTTTGTTTATTGCATAGTTGACTACTGAATATTAAGTTCGATGGCTTTTTGACGCATTATCTTAGTAGTTCGTTATTGGATTCG',
        'TGTTATGTTTATGCTTCTCCATTATGAATAATAGTGTGTACATTTGTGGTTTAACTTGATTATGTAACACATTGACCCGACATTTCGTTTTAAATGAGATATAAAAGGGATTCATGATTGCTTGTTCTTTCTTGGTTCCGTTAATGGGCATAATCAAGTATCTAGTTCAGTAGGGGGGAATGCCTTGGTGGGAAACGGTAAGGGGAATCGCTACCCATCTGCCATGACCTAGAGACTGTCACTCGATCACCATCAGGATCCGTGAATTATATTCCCAATTCGCCACCTCCGCTCTGCGGCTGAGTACAAGAGTTGCGCGTAGTACCAAACACAGCCACTTTAGTACGAAAATGGAGTGTAAATTTTTTTAGAACCCATCGCCTGGGTAAACAGCTTATGTCTCAGAGAAGCTAGGTTAAATGGTTCCAAAGTTGTCGGCTTTTAATAAGTAAGATGAGGATGTAAGACAAGAGAGAAAAAAGAGACAAACGACGCGAAAGAGGGAAGCATTTTCGTGTCGGACGCGAGATAATATAGGGAGGGTGTGAGTTAATTATCGAGTAAAACGCATACATGTTTTGGTGTCTGAAAAACATGGGGTAACAAGTTATCAGAGCACGCGAGCAAGTGGATGCTCTACCGATAACAATGCTATACAAAAACATAATATGGAAGACATGCAGTAGGGTTGAGGATGAGTAAATTATGGGAGGAGAATGGAGGGAGGGAATGATTGTACGAAAATTGCTATCGACTAAGAAGTCATTCCTTAAGAACCACGGTATTAAAACAGGCATGGGGGATGTAAATTCGCGTAAATAAACCTCAAATCACGTGTACGGAAATACAGTAGGATTTCCTAGAAAGCCTATGAGAAACGAAATTCTGAGTGACTAGAAACATTAAGTGATTACCAGTAATTTTGGTCCCTGGCAAATGTGGAGTAGCCTAACCAGTAGTCTGTGATTATATGCGGGTAGGTGAAAGTAACTTAATAGAT']

In [52]:
#print(np.expand_dims(encode_seq(strs[0]), 0).shape)
for i in range(4):
    s = strs[i]
    acts, ___, diff = eval_seqs(np.expand_dims(encode_seq(s), axis=0), model.model, i)
    print(diff)
    print(np.mean(acts[:,i]))

(1, 4)
-0.17986274
0.8111877
(1, 4)
-0.2241953
0.6832134
(1, 4)
-0.35910934
0.608433
(1, 4)
0.98086196
1.563827


In [53]:
rand_seqs = np.zeros((1000, 1000, 4))
for i in range(1000):
    raw_input_seq = np.eye(4)[np.random.choice(4, 1000)] # define an initial random image
    rand_seqs[i,:,:] = raw_input_seq
for i in range(4):
    activations, diffs, mean_diff = eval_seqs(rand_seqs, model.model, i)
    print(mean_diff)
    print(np.mean(activations[:,i]))




(1000, 4)
-0.15296823
0.6933241
(1000, 4)
-0.13878803
0.70395935
(1000, 4)
-0.51722485
0.42013165
(1000, 4)
0.8089812
1.4147861
